In [4]:
import gpxpy.gpx
import pandas as pd
import numpy as np

In [9]:
gpx_file = open("data/activity_10044912854.gpx", 'r')
points = []
gpx = gpxpy.parse(gpx_file)
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            # print(
            #     f'Point at with time {point.time} at ({point.latitude},{point.longitude}) elevation {point.elevation}')
            points.append([point.time,point.latitude,   point.longitude,  point.elevation])


In [12]:
point_dataframe = pd.DataFrame(np.array(points), columns = ['time', 'lat', 'lon', 'ele'])

In [ ]:
import geopandas as gpd


In [15]:
point_dataframe

,time,lat,lon,ele
0,2022-11-27 10:24:13+00:00,55.817366,37.434759,123.599998
1,2022-11-27 10:24:14+00:00,55.817355,37.434766,123.400002
2,2022-11-27 10:24:15+00:00,55.817346,37.434773,123.199997
3,2022-11-27 10:24:16+00:00,55.817334,37.434775,123.400002
4,2022-11-27 10:24:17+00:00,55.817323,37.434781,123.199997
...,...,...,...,...
4583,2022-11-27 11:40:36+00:00,55.817076,37.433549,122.199997
4584,2022-11-27 11:40:37+00:00,55.817071,37.433584,122.400002
4585,2022-11-27 11:40:38+00:00,55.817066,37.433606,122.599998
4586,2022-11-27 11:40:39+00:00,55.817063,37.433624,122.599998
